In [2]:
import nltk
import glob
import re
import scipy.io
import codecs
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import neural_network
from nltk.corpus import stopwords
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import FeatureUnion



BASE_DIR = 'C:/Users/Deepika_Bollavaram/Documents/IEOR290/Project/data/data/'
SPAM_DIR = 'spam/'
HAM_DIR = 'ham/'
TEST_DIR = 'test/'

def generate_text_list(filenames):
    text_list = []
    for fname in filenames:
        with codecs.open(fname, "r", "utf-8", errors = 'ignore') as f:
            text = f.read().replace('\r\n', ' ')
            #text = text.encode('utf-8')
            #text = ' '.join(str(r) for v in spam_list for r in v)
            text_list.append(text)
    return text_list

#reading files 
spam_filenames = glob.glob(BASE_DIR + SPAM_DIR + '*.txt')
spam_list = generate_text_list(spam_filenames)

ham_filenames = glob.glob(BASE_DIR + HAM_DIR + '*.txt')
ham_list = generate_text_list(ham_filenames)

test_filenames = [BASE_DIR + TEST_DIR + str(x) + '.txt' for x in range(5857)]
test_list = generate_text_list(test_filenames)

In [3]:
train_list = ham_list + spam_list

In [4]:
#Lemmatizing
from nltk.tag import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

wnl = WordNetLemmatizer()

#Parts-of-speech tagging the text and selecting only categories J, V, N and R
def get_wordnet_pos(treebank_tag):
    '''Treebank to wordnet POS tag'''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n' #basecase POS

train_list_clean=[]

for z in range(0,len(train_list)):
    train_list_split = train_list[z].split()
    toks = pos_tag(train_list_split)
    train_list_wnl = ' '.join([wnl.lemmatize(w,pos=get_wordnet_pos(t)) for w,t in toks])
    train_list_clean.append(train_list_wnl)

In [9]:
#### Training and Testing Split
y_ham = [0] * len(ham_list)
y_spam = [1] * len(spam_list)
y = y_ham + y_spam
text_train, text_test, y_train, y_test = train_test_split(train_list_clean,y, test_size=0.25, stratify=y)

In [ ]:
#### Method 1
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(text_train)
print(X_train_counts.shape)

#using tf-idf scores for prediction
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
print(X_train_tf.shape)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)

#building neural network
classifier = neural_network.MLPClassifier(activation = 'relu', hidden_layer_sizes = 200, batch_size = 100, early_stopping = True, validation_fraction = 0.2)
classifier.fit(X_train_tfidf, y_train)
X_new_counts = count_vect.transform(text_test)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

#predicting on test set
predicted = classifier.predict(X_new_tfidf)
print(confusion_matrix(y_test, predicted))

In [28]:
### Building on entire data set

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_list)
print(X_train_counts.shape)

tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
print(X_train_tf.shape)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)

classifier = neural_network.MLPClassifier(activation = 'relu', hidden_layer_sizes = 200, batch_size = 100, early_stopping = True, validation_fraction = 0.2)
classifier.fit(X_train_tfidf, y)
#X_new_counts = count_vect.transform(text_test)
#X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = classifier.predict(X_train_tfidf)
print(confusion_matrix(y, predicted))

(5172, 50463)
(5172, 50463)
(5172, 50463)
[[3663    9]
 [   2 1498]]


In [27]:
y_test.count(1)
float(11)/float(len(y_test))

0.008507347254447023

In [29]:
float(5161)/float(len(train_list))

0.9978731631863882

In [31]:
import pandas as pd
Kaggle_new_counts = count_vect.transform(test_list)
Kaggle_new_tfidf = tfidf_transformer.transform(Kaggle_new_counts)
predicted = classifier.predict(Kaggle_new_tfidf)
kaggle = pd.DataFrame(predicted)
kaggle.columns = ['Category']
kaggle.to_csv("C:/Users/Deepika_Bollavaram/Documents/IEOR290/Project/Submissions/NN_May2_New_v3.csv")